# Recomendação de filmes a partir de um perfil de usuário


Notebook com o intuito de prova de conceito e teste de viabilidade, para a implementação de sistema de recomendações baseados no perfil do usuário.
Há duas formas comumente usadas para implementação desse tipo de sistema de recomendações. A primeira é basear a recomendação a partir do que usuários "similares" assistiram, similares nesse caso sendo calculados a partir das avaliações dos filmes já assistidos. A outra forma é basear a recomendação a partir simplesmente das próprias avaliações do usuário. Para esse primeiro sprint, optou-se por usar a segunda forma, uma vez que a primeira tem muito risco de ter um desempenho ruim devido aos usuários mudarem preferências, bem como ter muitos usuários para conseguir ter similaridades boas o suficiente.


**Dataset**: ["The Movies Dataset"](https://www.kaggle.com/rounakbanik/the-movies-dataset), disponível no Kaggle em 18/09/2020

**Atenção:** Para execução desse notebook, é necessário download externo dos arquivos do dataset, que não estão disponíveis no repositório do MovieRec. 

In [2]:
import time
import pandas as pd
import ast
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression 

# Carregando e entendo os dados

Para a implementação, nós precisaremos dos dados contidos na tabela ratings retirados do TMDb(avaliações) apenas.
O ratings, como mostrado logo abaixo, basicamente se consiste no id do usuário que fez a avaliação, o id do filme que ele avaliou, seguido da avaliação em si, que vai de 1 a 5. Outra coluna originalmente colocada na tabela é o timestamp, mas foi retirado pelo não necessidade do uso.

In [3]:
dfRatings = pd.read_csv("ratings_small.csv", usecols=["userId", "movieId", "rating"])
dfRatingsFull = pd.read_csv("ratings.csv", usecols=["userId", "movieId", "rating"])
dfRatings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [4]:
nUsers = dfRatings.userId.unique().shape[0]
nMovies = dfRatings.movieId.unique().shape[0]
nUsersFull = dfRatingsFull.userId.unique().shape[0]
nMoviesFull = dfRatingsFull.movieId.unique().shape[0]
print(str(nUsers) + " users")
print(str(nMovies) + " movies")
print(str(dfRatings.shape[0]) + " ratings")
print("For full dataset")
print(str(nUsersFull) + " users")
print(str(nMoviesFull) + " movies")
print(str(dfRatingsFull.shape[0]) + " ratings")

671 users
9066 movies
100004 ratings
For full dataset
270896 users
45115 movies
26024289 ratings


# Pré processamento dos dados

Como comumente temos dados esparsos de avaliações, ou seja, cada usuário avalia muito poucos filmes no total, obviamente, usaremos um classificador Factorization Machines para treinar, uma vez que se comportam muito bem com tais estruturas de dados

Usaremos One-hot encoding nas tabelas userId e movieId, pois esses classificadores recebem como entrada tais estruturas

In [5]:
ratingsOneHot = pd.get_dummies(dfRatings, columns=['userId', 'movieId'], sparse=True)
ratingsOneHot.head()

,rating,userId_1,userId_2,userId_3,userId_4,userId_5,userId_6,userId_7,userId_8,userId_9,...,movieId_161084,movieId_161155,movieId_161594,movieId_161830,movieId_161918,movieId_161944,movieId_162376,movieId_162542,movieId_162672,movieId_163949
0,2.5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#No nosso modelo, o rating é o y, que queremos prever, e o restante das colunas é o X
y = ratingsOneHot['rating']
X = ratingsOneHot[ratingsOneHot.columns.difference(['rating'])]

In [7]:
print(y) # É preciso converter pra inteiro
y = y*10 # como é sabido que as notas só variam na primeira casa decimal, é só multiplicar por 10.
y

0         2.5
1         3.0
2         3.0
3         2.0
4         4.0
         ... 
99999     2.5
100000    4.0
100001    4.0
100002    2.5
100003    3.5
Name: rating, Length: 100004, dtype: float64


0         25.0
1         30.0
2         30.0
3         20.0
4         40.0
          ... 
99999     25.0
100000    40.0
100001    40.0
100002    25.0
100003    35.0
Name: rating, Length: 100004, dtype: float64

In [8]:
# Gasta muito menos tempo e ocupa menos espaço.
from scipy.sparse import lil_matrix
def data_frame_to_scipy_sparse_matrix(df):
    arr = lil_matrix(df.shape, dtype=np.float32)
    for i, col in enumerate(df.columns):
        ix = df[col] != 0
        arr[np.where(ix), i] = 1

    return arr.tocsr()

X_csr = data_frame_to_scipy_sparse_matrix(X)

In [9]:
# Dividindo o treinamento e teste.
start = time.time()
X_train, X_test, y_train, y_test = train_test_split(X_csr, y, test_size=0.3, random_state=42)
end = time.time()
duration = round(end-start, 2)
print("Train-test split: " + str(duration) + " secs")

Train-test split: 0.03 secs


In [10]:
# Treinando o modelo com o algorítmo de regressão logistica.
start = time.time()
model = LogisticRegression(random_state=0, multi_class='ovr', solver = 'liblinear')
model.fit(X_train, y_train)
end = time.time()
duration = round(end-start, 2)
print("Training: " + str(duration) + " secs")
print("\n")
model.score(X_test, y_test)

Training: 1.83 secs




0.3459769348710086

Resultado não tão bom, e sendo possível o uso do small_ratings apenas

# Usando o pacote Surprise para o Sistema de Recomendação

In [11]:
# Estamos usando o SVD pois se comporta melhor quando se trata de matrizes esparsas como é o caso.
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import GridSearchCV

In [12]:
reader = Reader()
ratingsDataSet = Dataset.load_from_df(dfRatings, reader)

In [13]:
param_grid = {
    "n_epochs": [5, 10, 20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.2, 0.4, 0.6]
}
# Estamos utilizando o GridSearchCV para testar vários parâmetros e escolher o melhor
ratingsGridSearch = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)
start = time.time()
ratingsGridSearch.fit(ratingsDataSet)
end = time.time()
duration = round(end-start, 2)
print("Training: " + str(duration) + "seconds")
print(ratingsGridSearch.best_score["rmse"])
print(ratingsGridSearch.best_params["rmse"])
print(ratingsGridSearch.best_score["mae"])
print(ratingsGridSearch.best_params["mae"])


Training: 294.26seconds
0.8950679211303569
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}
0.6916707645754588
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


In [14]:
#Carregar o dataset completo quebra em máquina "comum"
#ratingsDataSetFull = Dataset.load_from_df(dfRatingsFull, reader)
#trainset = ratingsDataSetFull.build_full_trainset()
#Treinando novamente a rede com os parametros escolhidos
trainset = ratingsDataSet.build_full_trainset()
svd = ratingsGridSearch.best_estimator['rmse']
start = time.time()
svd.fit(trainset)
end = time.time()
duration = round(end-start, 2)
print("Model data fitting time: " + str(duration) + "seconds")

Model data fitting time: 6.22seconds


In [15]:
svd.predict(1, 31)

Prediction(uid=1, iid=31, r_ui=None, est=2.526583523494994, details={'was_impossible': False})